# Продвинутый Python, семинар 9

**Лектор**: Петров Тимур

**Семинаристы**: Бузаев Федор, Дешеулин Олег, Коган Александра, Васина Олеся, Садуллаев Музаффар

Говорим про Flask, bootstrap, Jinja2 и про Docker.



## Jinja3

Jinja3 — это мощный шаблонизатор для Python, который позволяет создавать динамические HTML-страницы с использованием шаблонов. 

Jinja3 предоставляет мощные возможности для работы с шаблонами, включая условные операторы, циклы, фильтры и многие другие функции.



### Шаблонизатор

Шаблонизатор Jinja позволяет эффективно использовать наследование шаблонов, создавая один общий базовый шаблон для всего веб-приложения. 

Это помогает избежать дублирования кода и обеспечивает единообразный дизайн на всех страницах.

Шаг 1: Создайте базовый шаблон

Создайте файл, например, base.html, который будет содержать общую структуру страницы: шапку, навигацию, футер и места для динамического контента с помощью блоков.

```
<!-- base.html -->
<!DOCTYPE html>
<html lang="ru">
<head>
    <meta charset="UTF-8">
    <title>{% block title %}Название сайта{% endblock %}</title>
    <!-- Подключение стилей и скриптов -->
</head>
<body>
    <header>
        <!-- Общая шапка сайта -->
    </header>
    <nav>
        <!-- Общая навигация -->
    </nav>
    <main>
        {% block content %}
        <!-- Динамический контент страниц -->
        {% endblock %}
    </main>
    <footer>
        <!-- Общий футер сайта -->
    </footer>
</body>
</html>

```

Шаг 2: Создайте дочерние шаблоны для конкретных страниц

Для каждой страницы (главная, список, поиск и т.д.) создайте отдельные шаблоны, которые наследуют base.html и заполняют блоки конкретным содержимым.

Пример для главной страницы:

```
<!-- index.html -->
{% extends "base.html" %}

{% block title %}Главная страница{% endblock %}

{% block content %}
<h1>Добро пожаловать!</h1>
<p>Это главная страница вашего сайта.</p>
{% endblock %}
```


```
<!-- list.html -->
{% extends "base.html" %}

{% block title %}Список товаров{% endblock %}

{% block content %}
<h1>Наши товары</h1>
<ul>
    {% for item in items %}
    <li>{{ item.name }} - {{ item.price }} руб.</li>
    {% endfor %}
</ul>
{% endblock %}
```


Шаг 3: Используйте базовый шаблон в Flask-приложении

```
from flask import render_template

@app.route('/')
def index():
    return render_template('index.html')

@app.route('/list')
def item_list():
    items = get_items_from_database()
    return render_template('list.html', items=items)
```


### Условия 


Шаблонизатор позволяет отделить представление от логики приложения, храня шаблоны отдельно. Когда он получает данные, то подставляет их в соответствующие места в шаблоне. Это обеспечивает раздельное вычисление данных и их отображение, не смешивая логику с представлением. Если нам нужно изменить логику, мы не трогаем шаблон; если требуется обновить представление, работаем только с файлом шаблона. Такой подход особенно удобен при выводе больших блоков кода или сложных интерфейсов.

Взгляните на типичный шаблон интернет магазина. Вот какие блоки, использующие условия  и циклы могут быть на нем!

![image](pics/jinja_1.png)

В шаблонизаторе мы используем условные конструкции `if`, когда необходимо показать или скрыть определенный блок в зависимости от данных. 

Это позволяет динамически изменять отображение контента на основе условий. Примеры использования:

1. Отображение панели администратора только для администраторов: панель управления видна только пользователям с правами администратора.
2. Сообщение "нет товаров в корзине": если в корзине отсутствуют товары, выводится соответствующее уведомление.
3. Показать "нет доступа" при отсутствии прав: если пользователь не имеет необходимых разрешений, отображается сообщение об отсутствии доступа.
4. Возможность удаления комментария только для его автора: функция удаления комментария доступна исключительно тому пользователю, который его написал.


```
# предположим, в словаре user для обозначения является ли он администратором есть ключ is_admin, 
# значение которого может быть True или False

{% if user.is_admin %}
   <a href="#edit">Edit post</a>
{% endif %}
```

Мы применяем условные конструкции ```if...else``` в шаблонизаторе, когда нужно отображать разные блоки контента в зависимости от двух возможных условий. Это позволяет гибко изменять отображение страницы в соответствии с данными или состоянием пользователя. Примеры использования:

1. Разное отображение для обычных и про-пользователей: предоставление расширенных функций или эксклюзивного контента для про-пользователей, в то время как обычные пользователи видят стандартный контент.
2. Специальная цена для постоянных клиентов: если клиент является постоянным, ему отображается специальная цена или скидка; если клиент новый или обычный, показывается стандартная цена.
3. Отображение профиля для авторизованных пользователей и кнопки входа для неавторизованных: авторизованные пользователи видят доступ к своему профилю и персональным настройкам, а неавторизованные пользователи получают приглашение войти в систему или зарегистрироваться.

Использование ```if...else``` в шаблонизаторе помогает создавать динамичные и персонализированные интерфейсы, улучшая пользовательский опыт.

```
# предположим, в словаре user для обозначения того, залогинен ли пользователь, 
# есть ключ is_logged, значение которого может быть True или False

{% if user.is_logged %}
   <a href="#profile">{{ user.name }}</a>
{% else %}
   <a href="#auth">Login</a>
{% endif %}
```

### Наследование

Отдельный пункт - это наследование. Что это значит?

Опять-таки у вас есть страничка Вики ФКН. Вы видите, что у вас есть общие части (например header etc). Но мы же не хотим это прописывать для каждого сайта, верно?

Ровно поэтому есть наследование внутри Jinja2. Вы можете отдельно написать общую часть (которая отвечает отдельно за неизменяемые части, чаще всего это называется layout), а после просто добавлять другие блоки. Как это сделать?

Внутри родителя:

```
{% block body %}{% endblock %} - "резервируем" часть, которую может менять наследник
```

Внутри наследника:

```
{% extends "base.html" %} - указываем родителя, от чего наследуемся
{% block body %} - указываем часть, которую меняем
    <h1>Index</h1>
    <p class="important">
      Welcome to my awesome homepage.
    </p>
{% endblock %}
```

А еще поддерживается подобное наследование:

parent -> child -> grandchild

И если вам внутри child и grandchild вы меняете один и тот же блок (и хотите их оба вызвать), то тогда можно обращаться на уровень выше с помощью:

```
{{ super() }} - выведи то, что было на уровень выше

{{ super.super() }} - выведи то, что было на 2 уровня выше
```

### Циклы

Мы применяем цикл FOR в шаблонизаторе, когда нам нужно вывести элементы списка, например список тегов,  числовую последовательность,  адреса и так далее.

Если у нас есть список:
``` 
points = [
  "951 Marsh StreetMiddleburg, FL 32068",
  "207 Westport Drive Uniondale, NY 11553",
  "1 Second AvenueWayne, NJ 07470",
]
```

Вывод списка можно сделать так:
```
{% for point in points %}
   <p>{{ point }}</p>
{% endfor %}
```

И результат будет такой:

```
<p>951 Marsh StreetMiddleburg, FL 32068</p>
<p>207 Westport Drive Uniondale, NY 11553</p>
<p>1 Second AvenueWayne, NJ 07470</p>
```

### Фильтры



Создавая шаблоны, мы можем обходиться только переменными, условиями и циклами. Но, если нам хочется преобразовывать строки, списки и словари прямо в шаблоне, Jinja предоставляет нам набор фильтров. 

Фильтры отделены от переменной символом `(|)`, могут иметь необязательные аргументы в скобках. Важная фича: несколько фильтров могут быть связаны. Выход одного фильтра применяется к следующему.

Давайте взглянем на типичные фильтры.

```
 {{ mylist|length }}  – Этот возвращает длину списка. ({{ [433,217,202]|length }} -> 3) 

 {{ mylist|sort() }}  – Этот отсортирует список. ({{ [433,217,202]|sort() }} -> [202, 217, 433])

 {{ mylist|random }}  – Этот возвращает случайный элемент из списка. ({{ [433,217,202]|random }} -> 202)

 {{ myfloat|round|int }}  – Этот округлит число и превратит в натуральное. ({{ 123.456|round|int }} -> 123)
```


## JSON (JavaScript Object Notation)

JSON – текстовый формат для хранения и передачи структурированных данных. Несмотря на происхождение от JavaScript, он работает с любым языком программирования. Благодаря простому синтаксису JSON вы можете легко хранить и отправлять в другие приложения все, от чисел и строк до массивов и объектов. Структуры в формате JSON в виде файлов принято хранить с расширением .json.

Преимущества JSON

JSON широко распространен для обмена данными в Интернете благодаря компактности, гибкости, высокой читаемости даже для людей, далеких от программирования, наличию библиотек для работы с JSON у большинства языков программирования.

Основной синтаксис и структура

Текст JSON может быть построен на одной из двух структур:

– коллекция пар ключ-значение (объект),
– упорядоченный набор значений (массив).

Объекты пишутся в фигурных скобках {}, а их пары ключ: значение разделяются запятой.

Ключ и значение в паре разделяются двоеточием :

```
{
    "name": "John",
    "age": 30,
    "city": "New York"
}
```

Синтаксис JSON ужасно похож на синтаксис словарей и списков в Python, но обратите внимание на различие: ключи в JSON всегда являются строками! Значения могут быть любыми из семи типов значений, включая другой объект или массив.

Массивы пишутся в квадратных скобках `[ ]`, а их значения разделяются запятой. В массиве, также, значение может быть любого типа, включая другой массив или объект. Вот пример массива:

```
["night", "street", false, [ 345, 23, 8, "juice"], "fruit"]
```

JSON - это очень гибкий формат. Вы можете вкладывать объекты в другие объекты как свойства:

```
{
    "name": "John",
    "age": 30,
    "city": "New York",
    "children": [
        { "name": "Alice", "age": 6 },
        { "name": "Bob", "age": 8 }
    ]
}
```
Если объекты и массивы содержат другие объекты или массивы, данные имеют древовидную структуру.

Вложенные объекты полностью независимы и могут иметь разные свойства:

```
{
  "persons": [
    {
      "firstName": "Whitney",
      "age": 20
    },
    {
      "firstName": "Eugene",
      "lastName": "Lang"
    }
  ]
}
```

## Flask – обработка форм

##### Что такое форма?

Формы используются для сбора информации, которую пользователь вводит в специально отведённые поля. Когда он введёт свои данные и нажмет кнопку «Отправить», все эти данные будут отправлены на сервер. Затем они будут обработаны и сервер отправит ответ пользователю. Существует два основных метода отправки данных: GET и POST.

##### GET

Когда мы вводим в адресной строке браузера какой-либо адрес и переходим по нему, то отправляем серверу запрос, называемый GET. В таком запросе данные могут отсутствовать, как здесь: https://www.google.ru/. А вот запрос https://www.google.ru/search?q=stepik содержит в себе переменную q, которая имеет значение stepik. В данном случае запрос отправляется на адрес https://www.google.ru/search,  а данные из полей и их названия идут после ? через знак &. Предварительно данные кодируются в URL код, чтобы сервер не перепутал служебные символы (вроде /, или ﻿&) с частью запроса.

##### POST

Методом POST так же можно отправлять данные на сервер. Но, в отличие от GET, он может иметь тело - специальную "коробочку", в которую можно положить данные, которые уйдут на сервер.

Этот метод обычно используется для отправки форм и загрузки файлов. Хотя в жизни бывает по разному, на этом курсе мы будем передавать формы исключительно методом POST.

##### Создание формы

Элемент формы создаётся парным тегом `<form>`. Внутри размещаются сами поля, а те, что расположены за пределами формы, отправлены не будут. При необходимости в элементе формы можно использовать стороннюю разметку.

Форма имеет два обязательных атрибута: **action** и **method**.

В атрибуте **action** указывается ссылка на обработчик формы, то есть на тот роут, который обработает данные этой формы.

Атрибут **method** предназначен для указания метода отправки данных на сервер (GET или POST).

##### `<input>`

Поле задается одинарным тегом `<input>`. Данный тег стилизуется браузерами по-разному. Тип отображения по умолчанию inline (встроенный элемент), поэтому все поля отображаются в одной строке. Эта проблема решается подключением стилей, а пока мы до них не дошли - используй тег `<br>`.

##### name

Обязательный атрибут **name** – ключ для значения, которое будет отправлено на сервер , по этому ключу можно будет получить значение, которое ввел пользователь. 


#### Пример формы

Смотрите, это форма авторизации!

У нее есть action, method и три элемента input!

```
<form action="/auth/" method="POST">
        
   <input type="text" name="username">
   <input type="password" name="password">
	
   <input type="submit">
	
</form>
```

### Прием данных на сервере

Когда мы заполнили поля ввода и нажали отправить, на сервер передается POST запрос со всеми данными из форм. Чтобы вытащить из него данные, нужно использовать объект request -  глобальный объект запроса, предоставляемый Flask для доступа к данным входящего запроса. "Глобальный" означает, что он доступен в любом месте без его дополнительного создания или инициализации, достаточно импортировать.

##### Присматриваемся к форме

Атрибут action, как мы помним, означает адрес, на который будет отправлена форма.

Например, эта форма будет отправлена на /login/ значит нам нужно ловить ее по адресу site.me/login/:

```
<form action="/login/" method="POST" >
```

##### Получаем данные внутри роута

Создайте новый роут, который принимает запросы с методом POST:

```
@app.route('/search/', methods=['POST'])
def render_search():
    username = request.form['username']
    password = request.form['password']
    return f"Username: {username}, Password: {password}"
```


Давайте посмотрим на полный листинг кода 


In [2]:
from flask import Flask, render_template, request


app = Flask(__name__)
# вывод формы
@app.route("/login/")
def render_login():

    form= """
      <form action="/search/" method="post">     
        <input type="text" name="username">
        <input type="password" name="password">
        <input type="submit">
      </form>
    """

    # отдаем форму пользователю
    return form

# вывод результатов
@app.route('/search/', methods=['POST'])
def render_search():

    # вытаскиваем данные, которые пришли в request
    username = request.form.get("username")
    password= request.form.get("password")

    # отдаем результат пользователю
    return f"Логин: {username} Пароль {password}"

Давайте подробнее рассмотрим, что происходит в этом коде.

##### 1. Первый роут - отображение формы
```
@app.route("/login/")
def render_login():
```

- Этот роут обрабатывает GET-запросы по адресу `/login/`
- Когда пользователь заходит на эту страницу, он видит HTML-форму логинизации

##### 2. HTML-форма содержит:

```
<form action="/search/" method="post">     
    <input type="text" name="username">
    <input type="password" name="password">
    <input type="submit">
</form>
```

- `action="/search/"` - указывает, что данные формы будут отправлены на адрес `/search/`
- `method="post"` - определяет метод отправки данных как `POST` 
- Два поля ввода:
    Текстовое поле для имени пользователя (`name="username"`)
    Поле для пароля (`name="password"`)
    Кнопка отправки формы (`type="submit"`)

##### 3. Второй роут - обработка данных формы

```
@app.route('/search/', methods=['POST'])
def render_search():
```

- Этот маршрут принимает только POST-запросы (methods=['POST'])
- Активируется, когда пользователь отправляет форму

##### 4. Обработка данных:

In [ ]:
username = request.form.get("username")
password = request.form.get("password")

- `request.form.get()` извлекает данные из отправленной формы
- Значения получаются по именам полей (`username` и `password`)
- Метод `get()` безопасно возвращает `None`, если поле не найдено

## Flask сессии

Объект сессии — это способ хранить данные конкретных пользователей между запросами.

Так пользователь может добавить товар в корзину на одной странице, а на следующей увидеть, что товар все еще находится в корзине. Так же и со входом по паролю: пользователь может войти на одной странице, а Flask-приложение будет узнавать его на всех следующих.

Как это работает?

При первом обращении сервер отдает браузеру клиента cookie, которые тот передает при следующем запросе обратно серверу, а тот меняет и возвращает обратно браузеру и так далее. Данные сессии хранятся поверх cookie, и сервер подписывает их криптографически. Изменить их в браузере нельзя, для этого требуется SECRET_KEY, который есть только на стороне сервера.

Для получения данных из сессии и записи обратно в сессию используется объект session из пакета flask.

Мы можем работать с этим объектом как со словарем, содержимое которого индивидуально для каждого браузера-клиента.

Обратите внимание, поскольку сессии хранятся поверх куки, мы можем хранить ограниченный объем данных.

Записывать в сессию и читать из нее можно так, как будто это словарь:

```
session['username'] = 'admin'
print(session['username'])

>>> admin

print(session.get('username'))

>>> admin
```


Создадим простой код, чтобы показать работу сессий:

1. импортируем сессии;

2. создадим роут;

3. напишем его логику;

4. запустим приложение.


In [3]:
from flask import Flask, session

app = Flask(__name__)
app.secret_key = "randomstring"


@app.route('/') 
def increase():

  current = session.get("i", 0)     # начинаем с 0, увеличиваем i каждый раз
  session["i"] = current + 1
  return str(current)                # возвращаем i

# app.run()

Когда вы откроете этот пример в браузере, то увидите – 0.

После первого обновления страницы вы увидите – 1, после второго – 2, после третьего – 3.

##### Сохраняем простую настройку

Создадим еще один простой пример использования сессий.

Напишем переключатель темной темы на светлую.

Будем помнить для клиента, какую тему он использует, не привлекая для этого файлы и базы данных.

Создадим три роута:

1. роут, который записывает в сессию ключ `dark=True`;
2. роут, который записывает в сессию ключ `dark=False`;
3. роут, который выводит то, что записано в сессию.



In [4]:
from flask import Flask, session

app = Flask(__name__)
app.secret_key = "randomstring"

@app.route('/dark/')
def on():
    session['dark'] = True
    return "Set to Dark"


@app.route('/light/')
def off():
    session['dark'] = False
    return "Set to Light"


@app.route('/')
def show():
    dark = session.get("dark",False)

    return "<body bgcolor={}></body>".format("#000" if dark else "#fff")

# app.run()

Теперь, протестируем наше приложение:

Открываем `/`

Белый экран

Открываем `/dark`

Set to Dark

Открываем `/`

Черный экран

Открываем `/light`

Set to Light

Открываем `/`

Белый экран

Отлично, мы сохранили простую настройку и применили ее для стилизации приложения.

Там можно хранить размер шрифта, имя пользователя и другие штуки!

##### Хранение пользовательских данных

Попробуем сделать заготовку классического примера аутентификации.

Создадим для этого три роута:

1. роут, который имитирует аутентификацию и записывает в сессию данные пользователя;
2. роут, который как бы имитирует разлогинивание пользователя;
3. роут, который показывает статус пользователя.



In [ ]:
from flask import Flask, session

app = Flask(__name__)
app.secret_key = "randomstring"

@app.route('/login/')
def login():

    session['id'] = 1357
    session['name'] = "Mary Jane"
    return "User data set"

@app.route('/logout')
def logout():

    session.pop("id")
    session.pop("name")
    return "User data removed"

@app.route('/')
def index():
    return "Id: {} Name: {}".format(session.get("id"),session.get("name"))

# app.run()


Открываем `/`

Id: None Name: None


Открываем `/login`

User data set

Открываем `/`

Id: 1357 Name: Mary Jane

Открываем `/logout`

User data removed

Открываем `/`

Id: None Name: None

Скопируйте и запустите эту заготовку!

##### Хранение списков и словарей в сессии

Попробуем сделать еще один классический пример – корзину.

Создадим для этого три роута:

1. роут, который добавляет товар в корзину;
2. роут, который показывает корзину;
3. роут, который очищает корзину.

In [ ]:
# Импортируем объект сессии
from flask import Flask, session

app = Flask(__name__)
app.secret_key = "randomstring"


@app.route('/add/<item>/')
def add_to_cart(item):

    # Получаем либо значение из сессии, либо пустой список
    cart = session.get("cart", [])
    # Добавлям элемент в список
    cart.append(item)
    # Записываем список обратно в сессию
    session['cart'] = cart

    return "Item {} added".format(item)


@app.route('/')
def show_cart():
    return "You have {} in your cart".format(", ".join(session.get("cart", ["nothing"])))


@app.route('/reset/')
def reset_cart():
    # удаляем корзину из сессии
    session.pop("cart")
    return "Cart is empty!"

# app.run(debug=True)

Открываем `/`

You have nothing in your cart!

Открываем `/add/cream`

cream added

Открываем `/add/coffee`

coffee added

Открываем `/`

You have cream, coffee in your cart

Открываем `/reset`

Cart is empty!

Открываем `/`

You have nothing in your cart!

 

Отлично! Теперь мы можем хранить в сессиях еще и короткие списки!

##### Шпаргалка

Объект сессии — это способ хранить данные пользователей между запросами.

Сессии, с каждым клиентом, присваивается идентификатор сеанса.

Данные сессии хранятся поверх файлов cookie.

Для этого шифрования приложению требуется SECRET_KEY.

Запись данных в сессию:

```
session['username'] = 'admin'
```

Чтение данных из сессии:

```
username = session['username']
username = session.get('username')

```
Извлечение с удалением данных из сессии:

```
username = session.pop('username')
```

Удаление данных из сессии:

```
session.clear()
```

Работа со списками и словарями в сессии:

```
session['cart'] = ['item1', 'item2']
session['cart'].append('item3')
```



## Flask Аутентификация и авторизация

### Теория

Чтобы контролировать доступ пользователей к различным функциям и ресурсам вашего веб-приложения, необходимы два основных механизма: аутентификация и авторизация. Эти процессы обеспечивают безопасность системы и правильное распределение прав пользователей. Рассмотрим последовательность их работы:

1. Неизвестный пользователь с минимальными правами

    При первом посещении сайта пользователь не идентифицирован и рассматривается как гость. Ему доступны только ограниченные возможности, такие как просмотр общедоступных страниц. Он не может создавать сообщения, оставлять комментарии или получать доступ к персональным данным.

2. Предложение пройти аутентификацию

    Чтобы предоставить пользователю расширенные возможности, система предлагает ему представиться, то есть ввести свои учетные данные — логин и пароль — через форму входа на сайте.

3. Аутентификация пользователя

    Система проверяет предоставленные логин и пароль, сверяя их с информацией в базе данных. Если данные совпадают, пользователь считается аутентифицированным — его личность подтверждена, и мы знаем, кто он.

4. Выдача уникального идентификатора (сессии)

    После успешной аутентификации пользователю выдается уникальный идентификатор сессии. Этот идентификатор хранится, например, в cookies на стороне клиента и используется для отслеживания состояния пользователя между запросами.

5. Авторизация при последующих запросах

    При каждом новом запросе к серверу пользователь отправляет свой идентификатор сессии. Система использует этот идентификатор, чтобы определить, какие права и привилегии имеет пользователь, и решает, разрешить или запретить ему доступ к определенным ресурсам или действиям.

Различие между аутентификацией и авторизацией

1. Аутентификация — это процесс проверки подлинности пользователя. Она отвечает на вопрос: "Кто ты?" Здесь происходит идентификация личности пользователя с помощью предоставленных им учетных данных.

2. Авторизация — это процесс определения прав и привилегий аутентифицированного пользователя. Она отвечает на вопрос: "Что тебе разрешено делать?" На основе информации о пользователе система решает, к каким ресурсам и функциям он имеет доступ.

По-простому: 

1. Аутентификация подтверждает личность пользователя.
2. Авторизация определяет его права и доступы после аутентификации.

Три больших А (AAA): Authentication, Authorization, Accounting

В области информационной безопасности существует концепция трех больших А, которая включает:

1. Authentication (Аутентификация) — процесс установления подлинности пользователя.

2. Authorization (Авторизация) — процесс предоставления пользователю определенных прав и привилегий.

3. Accounting (Учет) — процесс записи и отслеживания действий пользователя в системе.

Учет (Accounting):

1. Назначение: Позволяет отслеживать, сколько ресурсов потребляет пользователь, какие действия выполняет, и вести статистику использования системы.
2. Применение: Учет важен для анализа поведения пользователей, биллинга, обеспечения безопасности и оптимизации ресурсов.
3. Пример: Ведение логов действий пользователя, подсчет количества запросов к API, отслеживание потраченных средств на счете.

### Простая аутентификация по жестко установленным логину и паролю

Сейчас мы создадим простое веб-приложение на Flask с авторизацией для одного пользователя. Информацию о пользователе (логин и пароль) будем хранить в настройках Flask (конфигурации приложения). Это поможет понять основные механизмы аутентификации и авторизации без необходимости подключения базы данных.

In [5]:
from flask import Flask

# Создаем приложение Flask
app = Flask(__name__)
# Настраиваем приложение Flask
# - Секретный код для сессий
app.config["SECRET_KEY"] = "secret_key"
# - Имя пользователя и пароль
app.config["USERNAME"] = "test"
app.config["PASSWORD"] = "test"


# ВНИМАНИЕ: Не храните секретные данные в коде приложения!
# Вместо этого используйте:
# - Переменные окружения 
# - Защищенные хранилища данных (например, базы данных)
# - Системы управления секретами

Пусть наша система будет состоять из двух страниц: секретной и публичной страницы аутентификации.

И давайте сразу ограничим доступ к приватной странице (сделаем авторизацию):

In [6]:
from flask import session, redirect, render_template

@app.route('/')
def home():
    # Авторизуем

    # - Если значение сессии is_auth не определено или равно False
    if not session.get('is_auth'):

        # - Делаем редирект на роут логина
        return redirect('/login')

    # Авторизованному пользователю показываем страницу
    return render_template("page_home.html")

Авторизация готова! Но теперь нам нужно реализовать часть связанную с аутентификацией. 

Что нам понадобится? Нам понадобится форма и функция, обрабатывающая два запроса. 

Один запрос на отображение формы (GET запрос) и второй - реализующий аутентификацию (POST запрос).

Воспользуемся шаблонизатором для отображения формы (файл page_login.html):

```
<form action="/login" method="post">
  Имя: <input type="text" name="username" value="">
  Пароль: <input type="password" name="password">
  <button type="submit">Войти</button>
</form>
```

А так выглядит полный код авторизации:

In [8]:
from flask import Flask, session, redirect, request, render_template

# Создаем приложение Flask
app = Flask(__name__)
# Настраиваем приложение Flask
# - Секретный код для сессий
app.config["SECRET_KEY"] = "secret_key"
# - Имя пользователя и пароль
app.config["USERNAME"] = "test"
app.config["PASSWORD"] = "test"

# Мы обрабатываем по данному URL и GET, и POST запрос
@app.route("/login", methods=["GET", "POST"])
def login():
    # Если пользователь авторизован
    if session.get("is_auth"):
        # то редиректим его на приватную страницу
        return redirect("/")

    # Переменная для хранения ошибок авторизации
    error_msg = ""  # Пока ошибок нет

    # Пришел запрос на аутентификацию
    if request.method == "POST":

        # Получаем отправленные данные
        username = request.form.get("username")
        password = request.form.get("password")

        # Проверяем полученные данные
        if ((username and password) and username == app.config["USERNAME"] and password == app.config["PASSWORD"]):

            # Устанавливаем в сессии признак, что пользователь аутентифицирован
            session["is_auth"] = True

            # Редиректим пользователя на приватную страницу
            return redirect("/")

        else:

            error_msg = "Неверное имя или пароль"

    # Отображаем форму аутентификации
    return render_template("page_login.html")

# app.run(debug=True, host='0.0.0.0', port=8080)

##### Разлогинивание

Нам же надо реализовать разлогинивание. Для этого достаточно удалить из сессии признак авторизации:

```
session.pop("is_auth")
```

In [9]:
@app.route('/logout/', methods=["POST"])
def logout():
    if session.get("is_auth"):
        session.pop("is_auth")
    return redirect("/login")

```
<form action="/logout/" method="post">
    <button type="submit">Выйти</button>
</form>
```



### Простая аутентификация из базы данных

Мы научились простой аутентификации и авторизации пользователя. Но что если нам нужно дать доступ к веб приложению нескольким пользователям? 

Для хранения их аккаунтов нам понадобится база данных (БД).


##### Подготовка базы данных

Настроим Flask-SQLAlchemy для работы с SQLite и создадим модель пользователя.

Файл БД test.db будет храниться в папке с приложением. Вот код приложения:



In [ ]:
from flask_sqlalchemy import SQLAlchemy

db = SQLAlchemy(app)

# Описываем модель пользователя
class User(db.Model):
 
  __tablename__ = 'users'
  id = db.Column(db.Integer, primary_key=True)
  username = db.Column(db.String(32), nullable=False, unique=True)
  password = db.Column(db.String(32), nullable=False)

Так добавляются пользователи в БД

In [ ]:
user = User(username = "test" ,password = "test")
db.session.add(user)
db.session.commit()

Теперь давайте реализуем авторизацию на основе БД:

In [ ]:
@app.route("/login", methods=["GET", "POST"])
def login():

    if request.method == "POST":

        username = request.form.get("username")
        password = request.form.get("password")

        # Ищем пользователя по имени
        user = User.query.filter_by(username=username).first()

        # Пользователь найден и пароль идентичен
        if user and user.password == password:

            # Сохраняем идентификатор пользователя в сессии
            session["user_id"] = user.id
            return "Вы зашли успешно"
        else:
            # Пользователь не найден или не верный пароль
            return "Неверное имя или пароль"

    # Если форма еще не отправлена:
    return render_template("page_login.html", error_msg=error_msg)

Выше в коде мы заменили имя значения сессии `"is_auth"` на `"user_id"` и сохранили в него идентификатор пользователя.
Зачем? Теперь мы знаем какой именно пользователь сделал запрос.

Использование идентификатора пользователя ещё один из простых способов авторизации (ранее мы рассмотрели просто проверку прохождения аутентификации - "залогинен" или нет). 

Теперь мы можем при создании постов, комментариев или чего-то ещё сохранять идентификатор создавшего их пользователя как владельца и потом, на основе этой минимальной информации, авторизовать запросы пользователя на действия с ними.

## CSS

CSS (Cascading Style Sheets) - это правила для внешнего вида нашего HTML-документа (которые мы видели абсолютно везде, на всех сайтах), без этого не было бы никакой красоты

Как можно задавать стиль? Структура любого CSS-документа выглядит как:

<для чего применить правило> - { <свойсто:значение; свойство:значение> }

Свойств достаточно [много](https://html5book.ru/css-spravochnik.html), давайте на примере разберемся, что внутри CSS документа:


```
Часть I - накидываем свойства на тэги

body { font-family: sans-serif; background: #eee; } 
a, h1, h2  { color: #377ba8; }
h1, h2 { font-family: 'Georgia', serif; margin: 0; }
h1 { border-bottom: 2px solid #eee; }
h2 { font-size: 1.2em; }
img {width: 35em;}

Часть II - накидываем свойства на классы (всегда с точкой)

.page { margin: 2em auto; width: 35em; border: 5px solid #ccc; padding: 0.8em; background: white; }
.entries  { list-style: none; margin: 0; padding: 0; }
.entries li {margin: 0.8em 1.2em}

Отдельный случай - здесь указывается форматирования для всех li внутри класса .entries


.entries li h2 { margin-left: -1em; }
.add_entry { font-size: 0.9em; border-bottom: 1px solid #ccc; }
.add_entry dl { font-weight: bold; }
.metanav { text-align: right; font-size: 0.8em; padding: 0.3em; margin-bottom: 1em; background: #fafafa;}
.flash { background: #cee5F5; padding: 0.5em; border: 1px solid #aacbe2; }
.border { background: #f0d6d6; padding: 0.5em; }

Если хотим для тэга с классом - то будет li.entries (как вариант)
```

А как решаются конфликты?

Допустим, что вы в одном месте указали, что текст должен быть красным, а в другом синим, кто победит?

Ответ: что позже идет в документе, то и выигрывает (на то оно и называется каскадированным)

## Docker
### Но... зачем?
Теперь мы перешли к самой сложной части - про докер.

(Повторение части с прошлого года)

Что такое докер? Представьте себе ситуацию: у вас есть приложение, которое вы хотите запустить и показать всему миру, какие вы классные. Что делать?

Можно, конечно, создать виртуалку, на нее накинуть приложение и там все делать. Но в какой-то момент вам придется класть туда несколько приложений и дальше начинается катавасия с загруженностью процесса. В какой-то момент одно приложение может стрельнуть (например, если у вас отдельное приложение для новостей, куда зайдет сразу дофига людей), сожрет все процессы и остальные части будут страдать. А еще же люди заходят с разными ОС, и надо конфигурации разные иметь, чтобы у всех все работало. Вот для этих вещей и есть контейнеры

Что такое контейнер? Это способ стандартизации развертки приложения и отделения его от общей инфраструктуры. Экземпляр приложения запускается в изолированной среде, не влияющей на основную операционную систему


![](https://d1.awsstatic.com/Developer%20Marketing/containers/monolith_2-VM-vs-Containers.78f841efba175556d82f64d1779eb8b725de398d.png)

Контейнер позволяют:

1. упаковать в единый образ приложение и все его зависимости: библиотеки, системные утилиты и файлы настройки. Это упрощает перенос приложения на другую инфраструктуру

2. приложения работают только внутри контейнеров и не имеют доступа к основной операционной системе. Это повышает безопасность приложений:они не смогут случайно или умышленно навредить основной системе. Если приложение в контейнере завершится с ошибкой или зависнет, это никак не затронет основную ОС (изоляция ресурсов)

3. избавляет от зависимости ОС: достаточно добавить необходимую конфигурацию в контейнер вместо процесса эмулирования одной ОС на другой (что трудозатратно)

4. За счет оптимизации контейнеров получаем также меньшую загрузку

### Ну что же, начнем

Первое, что надо сделать - это загрузить образ из Docker Hub. Образ (Image) - это схема нашего приложения, основа контейнера, с помощью которого его можно запустить. Все возможные образы хранятся [здесь](https://hub.docker.com/search?q=&type=image) (можно будет добавить сюда же и свои образы, которые вам необходимы)

Загрузим самый простой образ - busybox (дальше будет работа в терминале, а не в Колабе)

In [ ]:
docker pull busybox # загрузи образ busybox, если Permission denied, то запустите с sudo
docker images # посмотреть на все загруженные образы
docker rmi busybox # удалить образ

Отлично, загрузили, давайте запускать!

In [ ]:
docker run busybox # ничего не случилось, потому что мы ничего и не задали
docker run busybox echo "Hello" # о, что-то выдал
docker ps # посмотреть запущенные контейнеры (их пока нет, потому что прошлые кончились)
docker ps -a # посмотреть все контейнеры
docker run -it busybox sh #запустить на больше, чем 1 команду (-it - флаг интерактива)

Все запушенные и кончившиеся контейнеры занимают место, поэтому их еще стоит удалять (как минимум те, которые завершили свое действие)

In [ ]:
docker rm <ids> # удалить все ненужные контейнеры
docker rm $(docker ps -a -q -f status=exited) # чтобы не копировать все id
# -a все контейнеры, -q  - вывести только ID, -f - фильтр (на статус закончившися exited)

Обратите внимание сверху на значок Docker (MacOS). Видим, что Docker работает. Что это значит? Что у нас запушен процесс, с помощью которого это все вообще происходит (так называемый Docker Daemon)

### Запустим сайт из под капота

Окей, повеселились с каким-то образом. Как приложения-то запускать? Давайте попробуем запустить какой-нибудь более интересный образ

In [ ]:
docker pull prakhar1989/static-site
docker run prakhar1989/static-site #ничего не происходит, видим просто что is running
docker stop $(docker ps -a -q -f status=running) #выключаем все активные контейнеры
docker run -d -P --name static-site prakhar1989/static-site #давайте сделаем так:
# -d - открепляем наш терминал от контейнера(не будет прикреплен, можем продолжить работу в терминале)
# -P - сделаем порты открытыми и публичными, чтобы подключиться
# --name переименуем для удоства в static-site
docker port static-site #смотрим на порты, открываем на localhost:<порт>

Ура, запустили простенький сайт! А если поменять порт? Можно

In [ ]:
docker run -d -P -p 8888:80 prakhar1989/static-site --name static-site prakhar1989/static-site #теперь он на порту 8888
# окей, остановим все и удалим

### Хотим быть крутыми со своим образом!

Теперь хотим создать свой собственный образ и сделать еще интереснее. Образы делятся на 2 типа:

* Официальные образы - поддерживаются командой docker, скачать можно по названию (например, docker pull python)

* Неофициальные образы - образ, созданный пользователем, чаще всего выглядит как user/name

Скачаем простенький сайт, сделанный на Flask (о нем мы будет говорить в других семинарах) и запустим его локально

In [ ]:
git clone https://github.com/prakhar1989/docker-curriculum.git #скачаем для простого приложения на Flask (о нем мы будет говорить в других семинарах)
cd docker-curriculum/flask-app; pip install -r requirements.txt #установим все, что нужно для сайта
vim app.py #для того, чтобы у меня запустилась, так как порт 5000 у меня занят, я поменял в коде на 8888
python app.py

Получилось! Теперь создадим образ с этим приложением. Что нужно?

Так как сайт написан на Python, то надо скачать базовый образ python

In [ ]:
docker pull python:3-onbuild #Какой onbuild, це шо? Это надстройка при запуске возьмет на requirements.txt и установит за нас все, что нужно
# То есть такой помощник при запуске

Теперь надо как-то соединить приложение и образ. Это делается с помощью DockerFile - текстовый документ для автоматизации сборки (код для DockerFile почти идентичен аналогам bash)

Создаем там же, где и наше приложение файл Dockerfile и прописываем:

In [ ]:
vim Dockerfile

In [ ]:
FROM python:3-onbuild #указываем образ, который надо использовать

#благодаря тому, что у нас onbuild, нам не надо копировать файлы и устанавливать зависимости здесь

EXPOSE 8888 #говорим, на какой порт это все отправлять

CMD ["python", "./app.py"] # команды для запуска (то есть что надо сделать)

Осталось собрать образ через docker build:

In [ ]:
docker build -t palladain7/catgif . #здесь надо зарегаться на Docker Hub (это быстро) и в качестве user ввести свой ник
docker run -p 8888:8888 palladain/catgif #собрали-запустили
docker images #проверяем образ

Ура, мы собрали образ докера! Теперь осталось его загрузить на Docker Hub, чтобы его могли увидеть все и использовать)

In [ ]:
docker login #вначале надо авторизоваться
docker push palladain7/catgif #пушим
https://hub.docker.com/r/palladain7/catgif/ - проверяем, успех

## А теперь со всей этой информации можно уже и бота загрузить на docker!

### Делаем бота сильным и независимым

In [ ]:
%%writefile bot.py

from random import seed, randrange
from time import time
import telebot
from telebot import types

TOKEN = '5674479560:AAHI0lWyLHZQUa91Di-6NmNqdWbE7lL_6H8' # указываем токен нашего бота (для этого надо создать бота в @BotFather)
# Создайте собственного бота, чтобы наши наработки друг друга не перебивали

bot = telebot.TeleBot(TOKEN) # инициализируем нашего бота

parrots = {1: 'https://cindygurmann.files.wordpress.com/2018/06/ea2530ad-e913-4d5b-8036-762b5b227c04.jpeg',
           2: 'https://cherepah.ru/wp-content/uploads/2/2/8/228937ec782b8755993a3241e1d6c039.jpeg',
           3: 'https://kotsobaka.com/wp-content/uploads/2018/08/2748131046_8a253489b5_b.jpg',
           4: 'https://bestpopugai.ru/wp-content/uploads/2022/05/1-5.jpg',
           5: 'https://i.artfile.ru/1920x1200_952300_[www.ArtFile.ru].jpg',
           6: 'https://kipmu.ru/wp-content/uploads/pchppgks.jpg'}

favourite_parrot = 'https://pet7.ru/wp-content/uploads/2017/09/Popugaj-zhako-osobennosti-vida.jpg'

parrot_gif = 'https://i1.wp.com/cdn.dribbble.com/users/104127/screenshots/2589080/parrots.gif'

@bot.message_handler(commands=['start'])
def hello_message(message):
    markup = types.ReplyKeyboardMarkup(resize_keyboard=True, row_width=2) # указываем, сколько кнопок может быть в строке
    item_1 = types.KeyboardButton("Любимый попугай")
    item_2 = types.KeyboardButton("🎲")
    markup.add(item_1, item_2) #добавляем
    bot.send_message(message.chat.id, "Привет, тут будут попугаи!", reply_markup=markup)

@bot.message_handler(content_types=['text', 'emoji'])
def message_reply(message):
    if message.text=="Любимый попугай":
        markup = types.ReplyKeyboardMarkup()
        item = types.KeyboardButton(text='Хочу GIF-ку!')
        markup.add(item)
        bot.send_photo(message.chat.id, favourite_parrot, reply_markup = markup)
    elif message.text == "🎲":
        r = bot.send_dice(message.chat.id)
        bot.send_photo(message.chat.id, parrots[r.dice.value])
    elif message.text == 'Хочу GIF-ку!':
        markup = types.InlineKeyboardMarkup()
        item = types.InlineKeyboardButton(text='Ссылка', url=parrot_gif)
        item_1 = types.InlineKeyboardButton(text='Переслать', switch_inline_query=parrot_gif)
        markup.add(item, item_1)
        bot.send_animation(message.chat.id, parrot_gif, reply_markup = markup)
        markup = types.ReplyKeyboardMarkup(resize_keyboard=True, row_width=2) # указываем, сколько кнопок может быть в строке
        item_1 = types.KeyboardButton("Любимый попугай")
        item_2 = types.KeyboardButton("🎲")
        markup.add(item_1, item_2) #добавляем
        bot.send_message(message.chat.id, "Еще попуги!", reply_markup = markup)

@bot.inline_handler(func=lambda query: len(query.query) > 0)
def query_text(query):
    try:
        seed(int(time()))
        size = randrange(1, 100)
        print(size)
        r_sum = types.InlineQueryResultArticle(
                id='1', title="Папуг!",
                input_message_content = types.InputTextMessageContent(message_text="Ваш папуг " + str(size) + " размера" )
        )
        bot.answer_inline_query(query.id, [r_sum])
    except Exception as e:
        print("{!s}\n{!s}".format(type(e), str(e)))

bot.polling(none_stop=True, interval=0) #запускаем нашего бота

Writing bot.py


Вспоминаем, как работать с докером:

In [ ]:
docker pull python:3-onbuild

vim Dockerfile

In [ ]:
FROM python:3-onbuild #указываем образ, который надо использовать

#благодаря тому, что у нас onbuild, нам не надо копировать файлы и устанавливать зависимости здесь

EXPOSE 8888 #говорим, на какой порт это все отправлять

CMD ["python", "bot.py"] # команды для запуска (то есть что надо сделать)

In [ ]:
docker build -t palladain7/catgif . #здесь надо зарегаться на Docker Hub (это быстро) и в качестве user ввести свой ник
docker run -d -P -p 8888:8888 palladain/catgif #собрали-запустили
docker images #проверяем образ